In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import xarray as xr
import numpy as np
import datetime as dt
import database
import io
import os

/global/homes/t/taobrien/.conda/envs/climate-notebook/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
""" Load the artmip table"""
cmip6_data_table = database.load('/global/homes/t/taobrien/m1517_taobrien/artmip/tier2/cmip6_data_and_inventory/cmip6_list_20190909.txt', cache_file = "cmip6_list_20190909.pk")

Reading from cmip6_list_20190909.pk


In [3]:
cmip6_data_table['full_path'] = database.reconstruct_path(cmip6_data_table)

In [4]:

df_historical = database.select_by_dict(cmip6_data_table, 
                                        simulation = 'historical',
                                        group = '6hrLev')


df_ssp585 = database.select_by_dict(cmip6_data_table,
                            simulation = 'ssp585',
                            group = '6hrLev')

cmip6_6hr_hist_and_ssp = pd.concat([df_historical, df_ssp585])

In [5]:
""" Create a summary table"""

# create an empty table
#artmip_summary_table = pd.DataFrame(columns = ["Model", "Ensemble", "Variable", "historical sim. start year", "historical sim. end year", "ssp585 sim. start year", "ssp585 sim. end year", "Start Lat", "End Lat", "Start Lon", "End Lon", "Min Value", "Max Value"])
artmip_summary_table = pd.DataFrame(columns = ["Model", "Ensemble", "Variable", "historical sim. start year", "historical sim. end year", "ssp585 sim. start year", "ssp585 sim. end year"])

# loop over groups of model, ensemble, and variable
# Create a new table that gives the start/end dates for each run
for model, df2 in cmip6_6hr_hist_and_ssp.sort_values(by = 'model').groupby(by = 'model'):
    for ensemble, df1 in df2.sort_values(by = 'ensemble').groupby(by = 'ensemble'):
        for variable, df in df1.groupby(by = 'variable'):
            # get date-sorted versions of the historical and ssp585 file tables
            df_historical = database.select_by_dict(df, 
                                                    simulation = 'historical',
                                                    group = '6hrLev',
                                                    variable = variable).sort_values(by = 'startdate')
            df_ssp585 = database.select_by_dict(df,
                                                simulation = 'ssp585',
                                                group = '6hrLev',
                                                variable = variable).sort_values(by = 'startdate')
            
            # if there are files for the historical simulation, get the start and end years
            if len(df_historical) == 0:
                hist_start = np.nan
                hist_end = np.nan
            else:
                hist_start, hist_end = df_historical['startdate'].iloc[0].year, df_historical['enddate'].iloc[-1].year
                #sample_xr = xr.open_dataset(df_historical['full_path'].iloc[0])

            # if there are files for the ssp585 simulation, get the start and end years
               
           
            if False:
                
                if len(df_ssp585) == 0:
                    ssp585_start = np.nan
                    ssp585_end = np.nan

                    start_lat = np.nan
                    end_lat   = np.nan 
                    start_lon = np.nan 
                    end_lon   = np.nan 
                else:
                    ssp585_start, ssp585_end = df_ssp585['startdate'].iloc[0].year, df_ssp585['enddate'].iloc[-1].year
                    sample_xr = xr.open_dataset(df_ssp585['full_path'].iloc[0])

                    start_lat = float(sample_xr['lat'][0])
                    end_lat = float(sample_xr['lat'][-1])
                    start_lon = float(sample_xr['lon'][0])
                    end_lon = float(sample_xr['lon'][-1])

                    try:
                        min_val = float(sample_xr[variable].min())
                        max_val = float(sample_xr[variable].max())
                    except:
                        min_val = np.nan
                        max_val = np.nan
            else:
                
                if len(df_ssp585) == 0:
                    ssp585_start = np.nan
                    ssp585_end = np.nan
                else:
                    ssp585_start, ssp585_end = df_ssp585['startdate'].iloc[0].year, df_ssp585['enddate'].iloc[-1].year
                min_val = np.nan
                max_val = np.nan
                start_lat = np.nan
                end_lat   = np.nan 
                start_lon = np.nan 
                end_lon   = np.nan 



            # add a new row for this model-ensemble-variable combination
            new_column = {"Model" : model,
                          "Ensemble" : ensemble,
                          "Variable" : variable,
                          "historical sim. start year" : hist_start,
                          "historical sim. end year"   : hist_end,
                          "ssp585 sim. start year"      : ssp585_start,
                          "ssp585 sim. end year"        : ssp585_end,
                         }
                          #"Start Lat" : start_lat,
                          #"End Lat" : end_lat,
                          #"Start Lon" : start_lon,
                          #"End Lon" : end_lon,
                          #"Min Value" : min_val,
                          #"Max Value" : max_val,
                          #                                               }
            artmip_summary_table = artmip_summary_table.append(new_column, ignore_index = True)
            
artmip_summary_table

,Model,Ensemble,Variable,historical sim. start year,historical sim. end year,ssp585 sim. start year,ssp585 sim. end year
0,BCC-CSM2-MR,r1i1p1f1,hus,1950,2014,2015,2100
1,BCC-CSM2-MR,r1i1p1f1,ps,1950,2014,2015,2100
2,BCC-CSM2-MR,r1i1p1f1,ta,1950,2014,2015,2100
3,BCC-CSM2-MR,r1i1p1f1,ua,1950,2014,2015,2100
4,BCC-CSM2-MR,r1i1p1f1,va,1950,2014,2015,2100
5,CESM2,r10i1p1f1,hus,1850,2015,NaN,NaN
6,CESM2,r10i1p1f1,ps,1850,2015,NaN,NaN
7,CESM2,r10i1p1f1,ua,1850,2015,NaN,NaN
8,CESM2,r10i1p1f1,va,1850,2015,NaN,NaN
9,CESM2,r11i1p1f1,hus,1850,2015,NaN,NaN


In [6]:

artmip_summary_table_one_var = database.select_by_dict(artmip_summary_table, Variable = 'hus').drop(columns = 'Variable')
artmip_summary_table_one_var

,Model,Ensemble,historical sim. start year,historical sim. end year,ssp585 sim. start year,ssp585 sim. end year
0,BCC-CSM2-MR,r1i1p1f1,1950,2014,2015,2100
18,CNRM-CM6-1,r1i1p1f2,1950,2015,2015,2101
33,CNRM-ESM2-1,r1i1p1f2,1950,2015,2015,2101
106,IPSL-CM6A-LR,r1i1p1f1,1850,2015,2015,2055
224,MRI-ESM2-0,r1i1p1f1,1950,2014,2015,2100


In [7]:

artmip_summary_table_one_var = database.select_by_dict(artmip_summary_table, Variable = 'ua').drop(columns = 'Variable')
artmip_summary_table_one_var

,Model,Ensemble,historical sim. start year,historical sim. end year,ssp585 sim. start year,ssp585 sim. end year
3,BCC-CSM2-MR,r1i1p1f1,1950,2014,2015,2100
21,CNRM-CM6-1,r1i1p1f2,1950,2015,2015,2101
36,CNRM-ESM2-1,r1i1p1f2,1950,2015,2015,2101
110,IPSL-CM6A-LR,r1i1p1f1,1850,2015,2015,2055
227,MRI-ESM2-0,r1i1p1f1,1950,2014,2015,2100


In [8]:

artmip_summary_table_one_var = database.select_by_dict(artmip_summary_table, Variable = 'va').drop(columns = 'Variable')
artmip_summary_table_one_var

,Model,Ensemble,historical sim. start year,historical sim. end year,ssp585 sim. start year,ssp585 sim. end year
4,BCC-CSM2-MR,r1i1p1f1,1950,2014,2015,2100
22,CNRM-CM6-1,r1i1p1f2,1950,2015,2015,2101
37,CNRM-ESM2-1,r1i1p1f2,1950,2015,2015,2101
111,IPSL-CM6A-LR,r1i1p1f1,1850,2015,2015,2055
228,MRI-ESM2-0,r1i1p1f1,1950,2014,2015,2100
